1. Transformación de Datos
* Crea nuevas columnas: Basándonos en los datos existentes, crea nuevas columnas que sean útiles para el análisis. Por ejemplo, calcula el * ingreso total por venta y normaliza las ventas.
* Clasifica los datos: Crea una columna que clasifique las ventas en categorías significativas (e.g., ‘Alta’, ‘Media’, ‘Baja’).
2. Agrupación y Agregación
* Agrupación por múltiples columnas: Realiza agrupaciones por categorías como producto y tienda, producto y mes, etc.
* Aplicar funciones de agregación: Utiliza funciones como sum, mean, count, min, max, std, y var para obtener estadísticas descriptivas de cada grupo.
3. Análisis Personalizado con apply
* Función personalizada: Aplica funciones personalizadas para realizar análisis específicos que no se pueden lograr con las funciones de agregación estándar.
* Ejemplo de uso avanzado: Calcula la desviación de cada venta respecto a la media de su grupo.
4. Documentación
* Comentarios claros: Documenta claramente cada paso del análisis, explicando qué se hizo y por qué se hizo.
* Código legible: Asegúrate de que el código sea legible y esté bien comentado.

In [1]:
import pandas as pd
import numpy as np

In [16]:
import pandas as pd

# Ruta del archivo CSV en Google Drive
# Cambiar el enlace a formato de descarga directa
ruta_archivo = 'https://drive.google.com/uc?id=1d1r6n5uy4YqPLKZmDjno27FhLMaYD363'

# Cargar el archivo CSV en un DataFrame
df = pd.read_csv(ruta_archivo)




1. Transformación de Datos
* Crea nuevas columnas: Basándonos en los datos existentes, crea nuevas columnas que sean útiles para el análisis. Por ejemplo, calcula el ingreso total por venta y normaliza las ventas.

In [17]:
df['Total Sales'] = df['Quantity'] * df['Price per Unit']

In [18]:
df['Normalized Sales'] = (df['Total Sales'] - df['Total Sales'].min()) / (df['Total Sales'].max() - df['Total Sales'].min())
print(df[['Total Sales', 'Normalized Sales']].head())

   Total Sales  Normalized Sales
0          150          0.063291
1         1000          0.493671
2           30          0.002532
3          500          0.240506
4          100          0.037975


Clasifica los datos: Crea una columna que clasifique las ventas en categorías significativas (e.g., ‘Alta’, ‘Media’, ‘Baja’).

In [19]:
# Función para clasificar las ventas en 'Alta', 'Media', 'Baja' según los cuartiles
def classify_sales(sales):
    if sales >= df['Total Sales'].quantile(0.75):
        return 'Alta'
    elif sales >= df['Total Sales'].quantile(0.25):
        return 'Media'
    else:
        return 'Baja'

In [20]:
# Crear nueva columna con la clasificación de ventas
df['Sales Category'] = df['Total Sales'].apply(classify_sales)

# Mostrar las primeras filas para verificar
print(df[['Total Sales', 'Sales Category']].head())

   Total Sales Sales Category
0          150          Media
1         1000           Alta
2           30           Baja
3          500          Media
4          100          Media


2. Agrupación y Agregación
Agrupación por múltiples columnas: Realiza agrupaciones por categorías como producto y tienda, producto y mes, etc.

In [21]:
# Convertir la columna de fechas a formato datetime
df['Date'] = pd.to_datetime(df['Date'])

# Extraer el mes de la fecha
df['Month'] = df['Date'].dt.to_period('M')

# Agrupar por 'Product Category' y 'Month'
grouped_df = df.groupby(['Product Category', 'Month'])

# Mostrar la cantidad de elementos en cada grupo
print(grouped_df.size().head())

Product Category  Month  
Beauty            2023-01    25
                  2023-02    26
                  2023-03    21
                  2023-04    29
                  2023-05    28
dtype: int64


In [25]:
# Aplicar funciones de agregación para obtener estadísticas descriptivas por 'Product Category'
aggregated_stats = df.groupby('Product Category').agg({
    'Quantity': ['sum', 'mean', 'count', 'min', 'max', 'std', 'var'],
    'Price per Unit': ['mean', 'min', 'max', 'std', 'var'],
    'Total Amount': ['sum', 'mean', 'min', 'max', 'std', 'var']
})

# Mostrar el resultado de las estadísticas descriptivas
print(aggregated_stats)




                 Quantity                                              \
                      sum      mean count min max       std       var   
Product Category                                                        
Beauty                771  2.511401   307   1   4  1.100668  1.211471   
Clothing              894  2.547009   351   1   4  1.142647  1.305641   
Electronics           849  2.482456   342   1   4  1.153084  1.329603   

                 Price per Unit                                     \
                           mean min  max         std           var   
Product Category                                                     
Beauty               184.055375  25  500  194.929399  37997.470780   
Clothing             174.287749  25  500  186.017065  34602.348392   
Electronics          181.900585  25  500  189.049673  35739.778944   

                 Total Amount                                                   
                          sum        mean min   max        

Análisis Personalizado con apply
Función personalizada: Aplica funciones personalizadas para realizar análisis específicos que no se pueden lograr con las funciones de agregación estándar.

In [26]:
# Función personalizada para calcular la desviación respecto a la media de su grupo
def deviation_from_mean(x):
    return x - x.mean()

# Aplicar la función personalizada sobre 'Total Sales' para calcular la desviación de cada venta
df['Deviation from Group Mean'] = grouped_df['Total Sales'].transform(deviation_from_mean)

# Mostrar las primeras filas para verificar la desviación
print(df[['Total Sales', 'Deviation from Group Mean']].head())

   Total Sales  Deviation from Group Mean
0          150                -238.000000
1         1000                 558.787879
2           30                -365.800000
3          500                  28.243243
4          100                -344.642857
